<a href="https://www.kaggle.com/code/dorsaalh/apartment-rental-offers-in-germany-2?scriptVersionId=107759939" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import AutoTokenizer, AutoModelForMaskedLM
import seaborn as sns
import matplotlib.pyplot as plt
import altair as alt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/apartment-rental-offers-in-germany/immo_data.csv


In [2]:
pd.set_option('display.max_columns', None)
data = pd.read_csv('/kaggle/input/apartment-rental-offers-in-germany/immo_data.csv')
data.head(10)

,regio1,serviceCharge,heatingType,telekomTvOffer,telekomHybridUploadSpeed,newlyConst,balcony,picturecount,pricetrend,telekomUploadSpeed,totalRent,yearConstructed,scoutId,noParkSpaces,firingTypes,hasKitchen,geo_bln,cellar,yearConstructedRange,baseRent,houseNumber,livingSpace,geo_krs,condition,interiorQual,petsAllowed,street,streetPlain,lift,baseRentRange,typeOfFlat,geo_plz,noRooms,thermalChar,floor,numberOfFloors,noRoomsRange,garden,livingSpaceRange,regio2,regio3,description,facilities,heatingCosts,energyEfficiencyClass,lastRefurbish,electricityBasePrice,electricityKwhPrice,date
0,Nordrhein_Westfalen,245.00,central_heating,ONE_YEAR_FREE,NaN,False,False,6,4.62,10.0,840.00,1965.0,96107057,1.0,oil,False,Nordrhein_Westfalen,True,2.0,595.00,244,86.00,Dortmund,well_kept,normal,NaN,Sch&uuml;ruferstra&szlig;e,Schüruferstraße,False,4,ground_floor,44269,4.0,181.4,1.0,3.0,4,True,4,Dortmund,Schüren,Die ebenerdig zu erreichende Erdgeschosswohnun...,Die Wohnung ist mit Laminat ausgelegt. Das Bad...,NaN,NaN,NaN,NaN,NaN,May19
1,Rheinland_Pfalz,134.00,self_contained_central_heating,ONE_YEAR_FREE,NaN,False,True,8,3.47,10.0,NaN,1871.0,111378734,2.0,gas,False,Rheinland_Pfalz,False,1.0,800.00,NaN,89.00,Rhein_Pfalz_Kreis,refurbished,normal,no,no_information,NaN,False,5,ground_floor,67459,3.0,NaN,NaN,NaN,3,False,4,Rhein_Pfalz_Kreis,Böhl_Iggelheim,Alles neu macht der Mai – so kann es auch für ...,NaN,NaN,NaN,2019.0,NaN,NaN,May19
2,Sachsen,255.00,floor_heating,ONE_YEAR_FREE,10.0,True,True,8,2.72,2.4,1300.00,2019.0,113147523,1.0,NaN,False,Sachsen,True,9.0,965.00,4,83.80,Dresden,first_time_use,sophisticated,NaN,Turnerweg,Turnerweg,True,6,apartment,1097,3.0,NaN,3.0,4.0,3,False,4,Dresden,Äußere_Neustadt_Antonstadt,Der Neubau entsteht im Herzen der Dresdner Neu...,"* 9 m² Balkon\n* Bad mit bodengleicher Dusche,...",NaN,NaN,NaN,NaN,NaN,Oct19
3,Sachsen,58.15,district_heating,ONE_YEAR_FREE,NaN,False,True,9,1.53,40.0,NaN,1964.0,108890903,NaN,district_heating,False,Sachsen,False,2.0,343.00,35,58.15,Mittelsachsen_Kreis,NaN,NaN,NaN,Gl&uuml;ck-Auf-Stra&szlig;e,Glück-Auf-Straße,False,2,other,9599,3.0,86.0,3.0,NaN,3,False,2,Mittelsachsen_Kreis,Freiberg,Abseits von Lärm und Abgasen in Ihre neue Wohn...,NaN,87.23,NaN,NaN,NaN,NaN,May19
4,Bremen,138.00,self_contained_central_heating,NaN,NaN,False,True,19,2.46,NaN,903.00,1950.0,114751222,NaN,gas,False,Bremen,False,1.0,765.00,10,84.97,Bremen,refurbished,NaN,NaN,Hermann-Henrich-Meier-Allee,Hermann-Henrich-Meier-Allee,False,5,apartment,28213,3.0,188.9,1.0,NaN,3,False,4,Bremen,Neu_Schwachhausen,Es handelt sich hier um ein saniertes Mehrfami...,Diese Wohnung wurde neu saniert und ist wie fo...,NaN,NaN,NaN,NaN,NaN,Feb20
5,Schleswig_Holstein,142.00,gas_heating,NONE,NaN,False,True,5,4.48,2.4,NaN,1999.0,115531145,NaN,gas,True,Schleswig_Holstein,False,5.0,315.20,1e,53.43,Schleswig_Flensburg_Kreis,well_kept,NaN,no,Hardeseiche,Hardeseiche,False,2,apartment,24891,2.0,165.0,NaN,NaN,2,False,2,Schleswig_Flensburg_Kreis,Struxdorf,NaN,"helle ebenerdige 2 Zi. Wohnung mit Terrasse, h...",NaN,NaN,NaN,NaN,NaN,Feb20
6,Sachsen,70.00,self_contained_central_heating,ONE_YEAR_FREE,10.0,False,False,9,1.01,2.4,380.00,NaN,114391930,NaN,NaN,False,Sachsen,True,NaN,310.00,14,62.00,Mittelsachsen_Kreis,fully_renovated,NaN,NaN,Am Bahnhof,Am_Bahnhof,False,2,NaN,9599,2.0,NaN,1.0,4.0,2,True,3,Mittelsachsen_Kreis,Freiberg,Am Bahnhof 14 in Freiberg\nHeizkosten und Warm...,NaN,NaN,NaN,NaN,NaN,NaN,Feb20
7,Bremen,88.00,central_heating,ONE_YEAR_FREE,10.0,False,True,5,1.89,2.4,584.25,1959.0,115270775,NaN,gas:electricity,False,Bremen,True,2.0,452.25,35,60.30,Bremen,NaN,NaN,NaN,Lesumer Heerstr.,Lesumer_Heerstr.,False,3,ground_floor,28717,3.0,63.0,NaN,NaN,3,False,2,Bremen,St._Magnus,+ Komfortabler Bodenbelag: Die Wohnung ist zus...,Rollläden; Warmwasserbereiter; Kellerraum; Gas...,44.00,B,NaN,NaN,NaN,Feb20
8,Baden_Württemberg,110.00,oil_heating,ONE_YEAR_FREE,NaN,False,False,5,3.77,40.0,690.00,1970.0,106416361,1.0,oil,True,Baden_Württemberg,True,2.0,580.00,NaN,53.00,Emmen

In [3]:
data.shape

(268850, 49)

In [4]:
data.drop(columns=['scoutId', 'date']).duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
268845    False
268846    False
268847    False
268848    False
268849    False
Length: 268850, dtype: bool

### Column's descriptions:
**regio1:** Bundesland

**serviceCharge:** aucilliary costs such as electricty or internet in €

**heatingType:** Type of heating

**telekomTvOffer:** Is payed TV included if so which offer

**telekomHybridUploadSpeed:** how fast is the hybrid inter upload speed

**newlyConst:** is the building newly constructed

**balcony:** does the object have a balcony

**picturecount:** how many pictures were uploaded to the listing

**pricetrend:** price trend as calculated by Immoscout

**telekomUploadSpeed:** how fast is the internet upload speed

**totalRent:** total rent (usually a sum of base rent, service charge and heating cost)

**yearConstructed:** construction year

**scoutId:** immoscout Id

**noParkSpaces:** number of parking spaces

**firingTypes:** main energy sources, separated by colon

**hasKitchen:** has a kitchen

**geo_bln:** bundesland (state), same as regio1

**cellar:** has a cellar

**yearConstructedRange:** binned construction year, 1 to 9

**baseRent:** base rent without electricity and heating

**houseNumber:** house number

**livingSpace:** living space in sqm

**geo_krs:** district, above ZIP code

**condition:** condition of the flat

**interiorQual:** interior quality

**petsAllowed:** are pets allowed, can be yes, no or negotiable

**street:** street name

**streetPlain:** street name (plain, different formating)

**lift:** is elevator available

**baseRentRange:** binned base rent, 1 to 9

**typeOfFlat:** type of flat

**geo_plz:** ZIP code

**noRooms:** number of rooms

**thermalChar:** energy need in kWh/(m^2a), defines the energy efficiency class

**floor:** which floor is the flat on

**numberOfFloors:** number of floors in the building

**noRoomsRange:** binned number of rooms, 1 to 5

**garden:** has a garden

**livingSpaceRange:** binned living space, 1 to 7

**regio2:** District or Kreis, same as geo krs

**regio3:** City/town

**description:** free text description of the object

**facilities:** free text description about available facilities

**heatingCosts:** monthly heating costs in €

**energyEfficiencyClass:** energy efficiency class (based on binned thermalChar, deprecated since Feb 2020)

**lastRefurbish:** year of last renovation

**electricityBasePrice:** monthly base price for electricity in € (deprecated since Feb 2020)

**electricityKwhPrice:** electricity price per kwh (deprecated since Feb 2020)

**date:** time of scraping

# **Data pre-processing**

#### Drop columns which do not provide useful information

In [5]:
#'geo_bln' and 'regio1' are same columns due to columns description
#'street' and 'streetPlain' provide the same information
#'geo_krs' and 'regio2' provide the same information
data.drop(columns=['description', 'facilities', 'scoutId', 'geo_plz', 'geo_bln','geo_krs',
                   'houseNumber', 'street', 'yearConstructedRange', 'pricetrend', 'picturecount',
                  'streetPlain', 'firingTypes', 'date'], inplace=True)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268850 entries, 0 to 268849
Data columns (total 35 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   regio1                    268850 non-null  object 
 1   serviceCharge             261941 non-null  float64
 2   heatingType               223994 non-null  object 
 3   telekomTvOffer            236231 non-null  object 
 4   telekomHybridUploadSpeed  45020 non-null   float64
 5   newlyConst                268850 non-null  bool   
 6   balcony                   268850 non-null  bool   
 7   telekomUploadSpeed        235492 non-null  float64
 8   totalRent                 228333 non-null  float64
 9   yearConstructed           211805 non-null  float64
 10  noParkSpaces              93052 non-null   float64
 11  hasKitchen                268850 non-null  bool   
 12  cellar                    268850 non-null  bool   
 13  baseRent                  268850 non-null  f

In [7]:
data.describe()

,serviceCharge,telekomHybridUploadSpeed,telekomUploadSpeed,totalRent,yearConstructed,noParkSpaces,baseRent,livingSpace,baseRentRange,noRooms,thermalChar,floor,numberOfFloors,noRoomsRange,livingSpaceRange,heatingCosts,lastRefurbish,electricityBasePrice,electricityKwhPrice
count,261941.000000,45020.0,235492.000000,2.283330e+05,211805.000000,93052.000000,2.688500e+05,268850.000000,268850.000000,268850.000000,162344.000000,217541.000000,171118.000000,268850.000000,268850.000000,85518.000000,80711.000000,46846.000000,46846.000000
mean,151.206113,10.0,28.804928,9.013315e+02,1966.400590,1.327634,6.941294e+02,74.355548,3.765256,2.641261,114.749533,2.122405,3.572319,2.571542,3.070790,76.990866,2013.904536,89.113612,0.199769
std,308.295790,0.0,16.337151,3.323833e+04,46.992207,8.361403,1.953602e+04,254.759208,2.214357,2.633440,61.653663,3.634934,6.375496,0.937594,1.407127,147.716278,10.963125,5.395805,0.009667
min,0.000000,10.0,1.000000,0.000000e+00,1000.000000,0.000000,0.000000e+00,0.000000,1.000000,1.000000,0.100000,-1.000000,0.000000,1.000000,1.000000,0.000000,1015.000000,71.430000,0.170500
25%,95.000000,10.0,10.000000,4.698000e+02,1950.000000,1.000000,3.380000e+02,54.000000,2.000000,2.000000,79.000000,1.000000,2.000000,2.000000,2.000000,54.000000,2012.000000,90.760000,0.191500
50%,135.000000,10.0,40.000000,6.500000e+02,1973.000000,1.000000,4.900000e+02,67.320000,3.000000,3.000000,107.000000,2.000000,3.000000,3.000000,3.000000,70.000000,2017.000000,90.760000,0.198500
75%,190.000000,10.0,40.000000,9.850000e+02,1996.000000,1.000000,7.990000e+02,87.000000,5.000000,3.000000,140.300000,3.000000,4.000000,3.000000,4.000000,90.000000,2019.000000,90.760000,0.205500
max,146118.000000,10.0,100.000000,1.575154e+07,2090.000000,2241.000000,9.999999e+06,111111.000000,9.000000,999.990000,1996.000000,999.000000,999.000000,5.000000,7.000000,12613.000000,2919.000000,90.760000,0.227600


#### Delete data that do not make any sense

In [8]:
#due to data description, there are some buildings with minimum livingSpace of 0!!
data.drop(data[data.livingSpace == 0].index, inplace=True)

In [9]:
#due to data description, there are some buildings with maximum yearConstructed 2090!!
#since the test date was in 2019 maximum year of construction should be 2019.
data.drop(data[data.yearConstructed > 2019].index, inplace=True)

In [10]:
#due to data description, there are some buildings with minimum totalRent of 0!!
data.drop(data[data.totalRent == 0].index, inplace=True)

In [11]:
data.shape

(266386, 35)

#### Checking the percentage of null values

In [12]:
data.isnull().mean()*100

regio1                       0.000000
serviceCharge                2.541425
heatingType                 16.720849
telekomTvOffer              11.779523
telekomHybridUploadSpeed    83.187180
newlyConst                   0.000000
balcony                      0.000000
telekomUploadSpeed          12.055814
totalRent                   15.070987
yearConstructed             21.392641
noParkSpaces                65.658856
hasKitchen                   0.000000
cellar                       0.000000
baseRent                     0.000000
livingSpace                  0.000000
condition                   25.583552
interiorQual                42.002958
petsAllowed                 42.615603
lift                         0.000000
baseRentRange                0.000000
typeOfFlat                  13.669262
noRooms                      0.000000
thermalChar                 39.392461
floor                       19.013387
numberOfFloors              36.384420
noRoomsRange                 0.000000
garden      

#### Filling null values of totalRent using baseRent, serviceCharge, heatingCoost

In [13]:
data.totalRent.fillna((data.baseRent + data.serviceCharge + data.heatingCosts), inplace=True)

In [14]:
data.totalRent.isnull().mean()*100

10.477652729497796

In [15]:
#We want to predict totalRent so we drop rows without this value
data.dropna(subset=['totalRent'], axis=0, inplace=True)

#### Drop columns with more than 35% null values

In [16]:
data = data.loc[:, data.isnull().mean()*100 < 35]
data.shape

(238475, 24)

In [17]:
data.describe()

,serviceCharge,telekomUploadSpeed,totalRent,yearConstructed,baseRent,livingSpace,baseRentRange,noRooms,floor,noRoomsRange,livingSpaceRange
count,234437.000000,210505.000000,2.384750e+05,187707.000000,2.384750e+05,238475.000000,238475.000000,238475.000000,196732.000000,238475.000000,238475.000000
mean,148.611191,28.901634,8.881234e+02,1965.198874,6.431532e+02,73.358824,3.687271,2.618820,2.146850,2.555543,3.025101
std,319.467851,16.314571,3.252401e+04,47.078812,3.290126e+03,268.395987,2.204358,1.727827,3.789745,0.931862,1.384053
min,0.000000,1.000000,1.000000e+00,1000.000000,0.000000e+00,1.000000,1.000000,1.000000,-1.000000,1.000000,1.000000
25%,91.000000,10.000000,4.650000e+02,1945.000000,3.300000e+02,53.860000,2.000000,2.000000,1.000000,2.000000,2.000000
50%,130.110000,40.000000,6.390000e+02,1972.000000,4.789800e+02,66.480000,3.000000,3.000000,2.000000,3.000000,3.000000
75%,183.040000,40.000000,9.670000e+02,1996.000000,7.800000e+02,85.000000,5.000000,3.000000,3.000000,3.000000,4.000000
max,146118.000000,100.000000,1.575154e+07,2019.000000,1.234567e+06,111111.000000,9.000000,560.000000,999.000000,5.000000,7.000000


In [18]:
data.isnull().mean()*100

regio1                 0.000000
serviceCharge          1.693259
heatingType           16.353496
telekomTvOffer        11.461579
newlyConst             0.000000
balcony                0.000000
telekomUploadSpeed    11.728693
totalRent              0.000000
yearConstructed       21.288605
hasKitchen             0.000000
cellar                 0.000000
baseRent               0.000000
livingSpace            0.000000
condition             25.439564
lift                   0.000000
baseRentRange          0.000000
typeOfFlat            13.452563
noRooms                0.000000
floor                 17.504141
noRoomsRange           0.000000
garden                 0.000000
livingSpaceRange       0.000000
regio2                 0.000000
regio3                 0.000000
dtype: float64

#### Fill null values in condition, telekomTvOffer, heatingType, typeOfFlat, floor columns

In [19]:
data.condition.unique()

array(['well_kept', 'first_time_use', nan, 'refurbished',
       'fully_renovated', 'mint_condition',
       'first_time_use_after_refurbishment', 'modernized', 'negotiable',
       'need_of_renovation', 'ripe_for_demolition'], dtype=object)

In [20]:
data.condition.fillna('other', inplace=True)

In [21]:
data.telekomTvOffer.value_counts()

ONE_YEAR_FREE    203635
NONE               4335
ON_DEMAND          3172
Name: telekomTvOffer, dtype: int64

In [22]:
data.telekomTvOffer.fillna('unk', inplace=True)

In [23]:
data.heatingType.fillna('unk', inplace=True)

In [24]:
data.typeOfFlat.fillna('other', inplace=True)

#### Fill null values in serviceCharge and telekomUploadSpeed and yearConstructed and floor columns

In [25]:
data.serviceCharge.fillna(data.serviceCharge.mean(), inplace=True)

In [26]:
data.telekomUploadSpeed.fillna(data.telekomUploadSpeed.mean(), inplace=True)

In [27]:
data.yearConstructed.fillna(data.yearConstructed.mean(), inplace=True)

In [28]:
data.floor.fillna(data.floor.mean(), inplace=True)

#### Checking for duplicated rows

In [29]:
data.drop_duplicates(inplace = True)
data.shape

(233098, 24)

#### Delete outliers

In [30]:
for cols in data.columns:
    if data[cols].dtype == 'int64' or data[cols].dtype == 'float64':
        Q1 = data[cols].quantile(0.25)
        Q3 = data[cols].quantile(0.75)
        IQR = Q3 - Q1
        lower_range = Q1 - 1.5 * IQR
        upper_range = Q3 + 1.5 * IQR
        
        indexs = data[(data[cols] > upper_range) | (data[cols] < lower_range)].index
        data.drop(indexs, inplace=True)
data.shape

(168667, 24)

#### Performing sentiment analysis on description column

In [31]:
#tokenizer = AutoTokenizer.from_pretrained("bert-base-german-cased")

#model = AutoModelForMaskedLM.from_pretrained("bert-base-german-cased")

##### **To be continued ...**

# **Model**

In [32]:
new_data = data[['serviceCharge', 'heatingType', 'telekomUploadSpeed', 'totalRent']]

In [33]:
new_data

,serviceCharge,heatingType,telekomUploadSpeed,totalRent
0,245.00,central_heating,10.000000,840.00
2,255.00,floor_heating,2.400000,1300.00
3,58.15,district_heating,40.000000,488.38
4,138.00,self_contained_central_heating,28.901634,903.00
6,70.00,self_contained_central_heating,2.400000,380.00
...,...,...,...,...
268843,120.00,central_heating,40.000000,368.00
268844,80.00,gas_heating,2.400000,670.00
268845,90.00,heat_pump,10.000000,910.00
268847,220.00,central_heating,40.000000,930.00


#### One-hot categorical data

In [34]:
new_data.shape

(168667, 4)

In [35]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168667 entries, 0 to 268848
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   serviceCharge       168667 non-null  float64
 1   heatingType         168667 non-null  object 
 2   telekomUploadSpeed  168667 non-null  float64
 3   totalRent           168667 non-null  float64
dtypes: float64(3), object(1)
memory usage: 6.4+ MB


In [36]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168667 entries, 0 to 268848
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   serviceCharge       168667 non-null  float64
 1   heatingType         168667 non-null  object 
 2   telekomUploadSpeed  168667 non-null  float64
 3   totalRent           168667 non-null  float64
dtypes: float64(3), object(1)
memory usage: 6.4+ MB


In [37]:
categorical_cols = ['heatingType']

In [38]:
categorical_subset = new_data[categorical_cols]
categorical_subset = pd.get_dummies(categorical_subset)
categorical_subset.head()

,heatingType_central_heating,heatingType_combined_heat_and_power_plant,heatingType_district_heating,heatingType_electric_heating,heatingType_floor_heating,heatingType_gas_heating,heatingType_heat_pump,heatingType_night_storage_heater,heatingType_oil_heating,heatingType_self_contained_central_heating,heatingType_solar_heating,heatingType_stove_heating,heatingType_unk,heatingType_wood_pellet_heating
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [39]:
df = new_data.copy()

In [40]:
df.drop(columns=categorical_cols, inplace=True)
df = pd.concat([df, categorical_subset], axis=1)
df.shape

(168667, 17)

#### Split data

In [41]:
sep = int(0.8 * len(df))
train = df[:sep]
test = df[sep:]

In [42]:
l_train = train.totalRent
l_test = test.totalRent

train.drop(columns='totalRent', inplace=True)
test.drop(columns='totalRent', inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


#### Scaling

In [43]:
trans = StandardScaler()

s_train = trans.fit_transform(train)
s_test = trans.transform(test)

s_train = pd.DataFrame(s_train, columns=train.columns)
s_test = pd.DataFrame(s_test, columns=test.columns)

s_train

,serviceCharge,telekomUploadSpeed,heatingType_central_heating,heatingType_combined_heat_and_power_plant,heatingType_district_heating,heatingType_electric_heating,heatingType_floor_heating,heatingType_gas_heating,heatingType_heat_pump,heatingType_night_storage_heater,heatingType_oil_heating,heatingType_self_contained_central_heating,heatingType_solar_heating,heatingType_stove_heating,heatingType_unk,heatingType_wood_pellet_heating
0,2.178599,-1.213164,1.014821,-0.083262,-0.344393,-0.059059,-0.222061,-0.285265,-0.083892,-0.075212,-0.142515,-0.254434,-0.022945,-0.033692,-0.453382,-0.056936
1,2.363227,-1.705704,-0.985396,-0.083262,-0.344393,-0.059059,4.503271,-0.285265,-0.083892,-0.075212,-0.142515,-0.254434,-0.022945,-0.033692,-0.453382,-0.056936
2,-1.271175,0.731072,-0.985396,-0.083262,2.903663,-0.059059,-0.222061,-0.285265,-0.083892,-0.075212,-0.142515,-0.254434,-0.022945,-0.033692,-0.453382,-0.056936
3,0.203080,0.011810,-0.985396,-0.083262,-0.344393,-0.059059,-0.222061,-0.285265,-0.083892,-0.075212,-0.142515,3.930295,-0.022945,-0.033692,-0.453382,-0.056936
4,-1.052390,-1.705704,-0.985396,-0.083262,-0.344393,-0.059059,-0.222061,-0.285265,-0.083892,-0.075212,-0.142515,3.930295,-0.022945,-0.033692,-0.453382,-0.056936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134928,0.203080,0.731072,-0.985396,-0.083262,-0.344393,-0.059059,-0.222061,-0.285265,-0.083892,-0.075212,-0.142515,-0.254434,-0.022945,-0.033692,2.205644,-0.056936
134929,-0.867762,0.731072,-0.985396,-0.083262,-0.344393,-0.059059,-0.222061,-0.285265,-0.083892,-0.075212,-0.142515,-0.254434,-0.022945,-0.033692,2.205644,-0.056936
134930,0.424633,-1.213164,-0.985396,-0.083262,-0.344393,-0.059059,-0.222061,3.505518,-0.083892,-0.075212,-0.142515,-0.254434,-0.022945,-0.033692,-0.453382,-0.056936
134931,-0.867762,0.731072,1.014821,-0.083262,-0.344393,-0.059059,-0.222061,-0.285265,-0.083892,-0.075212,-0.142515,-0.254434,-0.022945,-0.033692,-0.453382,-0.056936


#### Multivariate linear regression

In [44]:
class LinearRegression:

  def __init__(self, learning_rate=0.1, epochs=100, accuracy_rate=0.1):
    self.learning_rate = learning_rate
    self.epochs = epochs
    self.accuracy_rate = accuracy_rate
    self.history = {'loss': [], 'accuracy': []}

  def gradient(self, X, y):
    return 2/X.shape[0] * np.dot(X.T, (np.dot(X, self.weights) - y))

  def fit(self, X, y, Xval, yval):
    train = []
    X = np.array(X)
    for i in range(len(X)):
      x = list(X[i])
      x.insert(0, 1)
      train.append(np.array(x))
    train = np.array(train)
    self.weights = np.random.rand((train.shape[1]))
    for i in range(self.epochs + 1):
      pred = self.predict(Xval)
      err = mean_squared_error(yval, pred)
      acc = self.calculate_accuracy(pred, yval)
      self.history['loss'].append(err)
      self.history['accuracy'].append(acc)
      self.weights = self.weights - self.learning_rate * self.gradient(train, y)

  def predict(self, X):
    pred = []
    X=np.array(X)
    for i in range(len(X)):
      x = list(X[i])
      x.insert(0, 1)
      pred.append(np.array(x))
    return np.dot(pred, self.weights)

  def calculate_accuracy(self, pred, val):
    right = 0
    val = np.array(val)
    for i in range(len(pred)):        
      if abs(pred[i] - val[i]) <= val[i] * self.accuracy_rate:
        right += 1
    return right / len(pred)

In [45]:
model = LinearRegression()
model.fit(s_train, l_train, s_test, l_test)
y_pred = model.predict(s_test)

In [46]:
print(model.calculate_accuracy(y_pred, l_test))

0.26228730657496885


In [47]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(s_train, l_train)
reg.score(s_test, l_test)

0.40070036170371337

In [48]:
from sklearn.linear_model import Lasso

reg_lasso = Lasso(alpha=1.0)
reg_lasso.fit(s_train, l_train)
reg_lasso.score(s_test, l_test)

0.4005520693431519

In [49]:
from sklearn.linear_model import Ridge

reg_ridge = Ridge(alpha=1.0)
reg_ridge.fit(s_train, l_train)
reg_ridge.score(s_test, l_test)

0.40071895651659784